In [1]:
import numpy as np
import numpy.linalg as lia
import pandas as pd
import matplotlib as plt
import cv2 as cv
from sklearn import svm 
from sklearn import datasets
from sklearn.datasets import fetch_openml
from sklearn.naive_bayes import GaussianNB

In [2]:
digits = datasets.load_digits()
wine = fetch_openml(name='wine', version=1)

In [3]:
# normalization of digits and wine data

digits_data_norm = []

for col in digits.data:
    col_norm = col/np.max(col)
    digits_data_norm.append(col_norm)

digits.data = np.asarray(digits_data_norm)

wine_data_norm = []

for col in wine.data.T:
    col_norm = col/np.amax(col)
    wine_data_norm.append(col_norm)
    
wine.data = np.asarray(wine_data_norm).T

In [4]:
# 5-fold cross validation for digits dataset

digitsTrainingSetSize = int(np.ceil(0.8 * len(digits.data)))
digitsValidationSetSize = int(len(digits.data) - digitsTrainingSetSize)

xDigitsTrainingSets = []
yDigitsTrainingSets = []
xDigitsValidationSets = []
yDigitsValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(digits.data[foldIndex*digitsValidationSetSize:((foldIndex*digitsValidationSetSize)+digitsValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(digits.target[index+(foldIndex*digitsValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(digits.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(digits.target[index])
            
    xDigitsTrainingSets.append(xTrainingSet)
    yDigitsTrainingSets.append(yTrainingSet)
    xDigitsValidationSets.append(xValidationSet)
    yDigitsValidationSets.append(yValidationSet)
    
# 5-fold cross validation for wine dataset

wineTrainingSetSize = int(np.ceil(0.8 * len(wine.data)))
wineValidationSetSize = int(len(wine.data) - wineTrainingSetSize)

xWineTrainingSets = []
yWineTrainingSets = []
xWineValidationSets = []
yWineValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []
    for index, data in enumerate(wine.data[foldIndex*wineValidationSetSize:((foldIndex*wineValidationSetSize)+wineValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(wine.target[index+(foldIndex*wineValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []
    
    for index, data in enumerate(wine.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(wine.target[index])
            
    xWineTrainingSets.append(xTrainingSet)
    yWineTrainingSets.append(yTrainingSet)
    xWineValidationSets.append(xValidationSet)
    yWineValidationSets.append(yValidationSet)

In [5]:
# one-hot encoding of y for digits dataset

numberOfDigitsTargets = 10
numberOfWineTargets = 3

for index, fold in enumerate(yDigitsTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yDigitsValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsValidationSets[index] = encodedFold

# one-hot encoding of y for wine dataset

for index, fold in enumerate(yWineTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yWineValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineValidationSets[index] = encodedFold


In [6]:
def getRandomIndices(arr, batch_size):
    indices = []
    
    if batch_size > len(arr):
        print("Error: batch size larger than size of dataset.")
        return
    
    while batch_size > 0:
        x = np.floor(np.random.random() * len(arr))
        if x not in indices:
            indices.append(int(x))
            batch_size -= 1
    
    return indices

In [7]:
# gradient descent class
 
class GradientDescent:
    
    def __init__(self, batch_size, learning_rate=0.5, momentum=0.9, max_termination_condition=25, max_iters=1000):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_termination_condition = max_termination_condition
        self.max_iters = max_iters
        self.deltas = []
        
    def run(self, gradient_fn, x, y, w):
        t = 1
        
        max_accuracy = -1
        termination_count = 0        
        weight_history = []
        error_history = []
                
        for number_of_targets in range(len(y[0])):
            weight_history.append([])
        
        while termination_count < self.max_termination_condition and t < self.max_iters:
            gradients = gradient_fn(x, y, w, self.batch_size)   
            
            for c in range(len(y[0])):
                if(t==1):
                    w[c] = w[c] - self.learning_rate * gradients[c]
                else:
                    delta_w = (self.momentum)*(self.deltas[-(len(y[0]))]) + (1-self.momentum)*gradients[c]
                    w[c] = w[c] - (self.learning_rate)*(delta_w)
                self.deltas.append(w[c])
            
            a = np.asarray(x)
            b = np.asarray(w)
    
            yh=[]
            for i, x_c in enumerate(a):
                yh_x=[]

                for c in range(len(b)):
                    w_x =  b[c] @ x_c
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  b[i] @ x_c
                        den += np.exp(w_x)

                    yh_c = num/den
                    yh_x.append(yh_c)
                    
                yh.append(yh_x)
                
            step_accuracy = 0
                
            def accurate(a, b):
                return np.argmax(a) == np.argmax(b)
                
            for sample_index, yh_x in enumerate(yh):
                if accurate(yh_x, y[sample_index]):
                    step_accuracy += 1
                    
            step_accuracy /= len(x)
            
            for c in range(len(b)):
                weight_history[c].append(w[c])
            
            error_history.append(step_accuracy)
            
            # We use an alternate termination condition that terminates faster as
            # the suggested condition ran for a longtime for us (~1hr).
            
            # We track the best training accuracy encountered, and if the 
            # next max_termination_condition-number of steps do not have a better accuracy, then
            # it terminates.

            if step_accuracy > max_accuracy:
                max_accuracy = step_accuracy
                termination_count = 0
                print(f"\t\tStep {t}: new best accuracy of {max_accuracy:.3f}")
            else:
                termination_count += 1
                print(f"\t\tStep {t}")
            
            t += 1
        
        # take the weight prior to the last max_termination_condition-number of weights
        # (as it is guaranteed to be the best prior to termination).
        
        index_best = len(error_history)-self.max_termination_condition-1
        
        w_best = []
        
        for c in range(len(y[0])):
            w_best.append(weight_history[c][index_best])
        
        return w_best

In [8]:
# logistic regression

class LogisticRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, optimizer):
        def gradient(x, y, w, batch_size):
            gradients = np.zeros(len(w)).tolist()

            indices = getRandomIndices(x, batch_size)

            for index in indices:
                a = np.asarray(x[index])
                b = np.asarray(y[index])

                for c in range(len(b)):
                    w_x =  w[c] @ a
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  w[i] @ a
                        den += np.exp(w_x)

                    yh_c = num/den

                    y_c = b[c]
                    
                    cost_c = np.dot(yh_c - y_c, a)
                    
                    gradients[c] += cost_c

            return gradients
        
        if self.add_bias:
            x = np.asarray(x)
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])

        w0 = []
        for c in range(len(y[0])):
            w0.append(np.zeros(len(x[0])))
            
        self.w = optimizer.run(gradient, x, y, w0)
        return self
    
    def predict(self, x):
        if self.add_bias:
            x = np.asarray(x)
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])

        a = np.asarray(x)
        b = np.asarray(self.w)

        yh=[]
        
        for i, x_c in enumerate(a):
            yh_x=[]
            
            for c in range(len(b)):
                w_x =  b[c] @ x_c
                num = np.exp(w_x)

                den = 0
                for i in range(len(b)):
                    w_x =  b[i] @ x_c
                    den += np.exp(w_x)

                yh_c = num/den
                yh_x.append(yh_c)
                
            yh.append(yh_x)
        
        return yh

In [9]:
def runLogisticRegression(batch_size, learning_rate, momentum):
    def accurate(a, b):
        return np.argmax(a) == np.argmax(b)

    def cost(yh, y):
        return y * np.log1p(np.exp(-yh)) + (1-yh) * np.log1p(np.exp(yh))

    print("Model hyper-parameters:")
    print("\tMini-batch size:", batch_size)
    print("\tLearning rate:", learning_rate)
    print("\tMomentum:", momentum)
    print("\n")

    digits_training_accuracy = 0
    digits_training_cost = 0
    digits_validation_accuracy = 0
    digits_validation_cost = 0

    print("Digits gradient descent:")

    for fold_index, fold in enumerate(xDigitsTrainingSets):
        print(f"\tCross-validation fold {fold_index+1}")

        gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
        logisticRegressionModel = LogisticRegression(add_bias=True)

        logisticRegressionModel.fit(fold, yDigitsTrainingSets[fold_index], gradientDescentModel)
        yh_training = logisticRegressionModel.predict(xDigitsTrainingSets[fold_index])
        yh_validation = logisticRegressionModel.predict(xDigitsValidationSets[fold_index])

        for sample_index, yh_x in enumerate(yh_training):
            if accurate(yh_x, yDigitsTrainingSets[fold_index][sample_index]):
                digits_training_accuracy += 1
            c = np.argmax(yDigitsTrainingSets[fold_index][sample_index])
            cst = cost(yh_x[c], yDigitsTrainingSets[fold_index][sample_index][c])
            digits_training_cost += cst

        for sample_index, yh_x in enumerate(yh_validation):
            if accurate(yh_x, yDigitsValidationSets[fold_index][sample_index]):
                digits_validation_accuracy += 1
            c = np.argmax(yDigitsValidationSets[fold_index][sample_index])
            cst = cost(yh_x[c], yDigitsValidationSets[fold_index][sample_index][c])
            digits_validation_cost += cst

    digits_training_accuracy /= 4*len(digits.data)
    digits_training_cost /= 4
    digits_validation_accuracy /= len(digits.data)

    wine_training_accuracy = 0
    wine_training_cost = 0
    wine_validation_accuracy = 0
    wine_validation_cost = 0

    print("Wine gradient descent:")

    for fold_index, fold in enumerate(xWineTrainingSets):
        print(f"\tCross-validation fold {fold_index+1}")

        gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
        logisticRegressionModel = LogisticRegression(add_bias=True)

        logisticRegressionModel.fit(fold, yWineTrainingSets[fold_index], gradientDescentModel)
        yh_training = logisticRegressionModel.predict(xWineTrainingSets[fold_index])
        yh_validation = logisticRegressionModel.predict(xWineValidationSets[fold_index])

        for sample_index, yh_x in enumerate(yh_training):
            if accurate(yh_x, yWineTrainingSets[fold_index][sample_index]):
                wine_training_accuracy += 1
            c = np.argmax(yWineTrainingSets[fold_index][sample_index])
            cst = cost(yh_x[c], yWineTrainingSets[fold_index][sample_index][c])
            wine_training_cost += cst

        for sample_index, yh_x in enumerate(yh_validation):
            if accurate(yh_x, yWineValidationSets[fold_index][sample_index]):
                wine_validation_accuracy += 1
            c = np.argmax(yWineValidationSets[fold_index][sample_index])
            cst = cost(yh_x[c], yWineValidationSets[fold_index][sample_index][c])
            wine_validation_cost += cst

    wine_training_accuracy /= 4*len(wine.data)
    wine_training_cost /= 4
    wine_validation_accuracy /= len(wine.data)

    print("\n")
    print(f"Digits training accuracy: {digits_training_accuracy*100:.1f}%")
    print(f"Digits training cost: {digits_training_cost:.3f}")
    print(f"Digits validation accuracy: {digits_validation_accuracy*100:.1f}%")
    print(f"Digits validation cost: {digits_validation_cost:.3f}")
    print(f"Wine training accuracy: {wine_training_accuracy*100:.1f}%")
    print(f"Wine training cost: {wine_training_cost:.3f}")
    print(f"Wine validation accuracy: {wine_validation_accuracy*100:.1f}%")
    print(f"Wine validation cost: {wine_validation_cost:.3f}")
    
    return (digits_validation_accuracy, digits_validation_cost, wine_validation_accuracy, wine_validation_cost)

#runLogisticRegression(30, 0.04, 0.2)


In [10]:
euclidean = lambda x1, x2: np.sqrt(np.sum((x1 - x2)**2, axis=-1))
manhattan = lambda x1, x2: np.sum(np.abs(x1 - x2), axis=-1)

class KNN:

    def __init__(self, K=1, dist_fn= euclidean):
        self.dist_fn = dist_fn
        self.K = K
        return
    
    def fit(self, x, y):
        self.x = x
        self.y = y
        self.C = len(y[0])
        return self
    
    def predict(self, x_test):
        num_test = x_test.shape[0]
        distances = self.dist_fn(self.x[None,:,:], x_test[:,None,:])
        knns = np.zeros((num_test, self.K), dtype=int)
        y_prob = np.zeros((num_test),dtype=int)
        counts = np.zeros((num_test, self.C))
        
        for i in range(num_test):
            knns[i,:] = np.argsort(distances[i])[:self.K]
            k_count=np.zeros(self.K, dtype=int)
            
            for s, arr in enumerate(self.y[knns[i,:]]):
                k_count[s] = np.argmax(arr)
            
            y_prob_i, counts_i = np.unique(k_count, return_counts=True)
            y_prob[i] = int(y_prob_i[np.argmax(counts_i)])
        
        return y_prob, knns

In [11]:
KNNmodel = KNN(K=11)

digits_knn_accuracy = 0

for fold in range(5):
    y_prob, knns = KNNmodel.fit(np.asarray(xDigitsTrainingSets[fold]), np.asarray(yDigitsTrainingSets[fold])).predict(np.asarray(xDigitsValidationSets[fold]))
    
    for i, prob in enumerate(y_prob):
        if prob == np.argmax(yDigitsValidationSets[fold][i]):
            digits_knn_accuracy += 1

digits_knn_accuracy /= len(digits.data)

print(f"KNN digits validation accuracy: {digits_knn_accuracy*100:.1f}%")

KNNmodel = KNN(K=7)

wine_knn_accuracy = 0

for fold in range(5):
    y_prob, knns = KNNmodel.fit(np.asarray(xWineTrainingSets[fold]), np.asarray(yWineTrainingSets[fold])).predict(np.asarray(xWineValidationSets[fold]))
    
    for i, prob in enumerate(y_prob):
        if prob == np.argmax(yWineValidationSets[fold][i]):
            wine_knn_accuracy += 1
            
wine_knn_accuracy /= len(wine.data)

print(f"KNN wine validation accuracy: {wine_knn_accuracy*100:.1f}%")

KNN digits validation accuracy: 95.7%
KNN wine validation accuracy: 85.4%


In [12]:
digits_naive_accuracy = 0

for fold in range(5):
    labels_training = np.zeros(len(yDigitsTrainingSets[fold]))
    
    for i, arr in enumerate(yDigitsTrainingSets[fold]):
        labels_training[i] = np.argmax(arr)
        
    labels_validation = np.zeros(len(yDigitsValidationSets[fold]))
    
    for i, arr in enumerate(yDigitsValidationSets[fold]):
        labels_validation[i] = np.argmax(arr)
    
    gnb = GaussianNB()
    y_pred = gnb.fit(np.asarray(xDigitsTrainingSets[fold]), labels_training).predict(np.asarray(xDigitsValidationSets[fold]))

    for i, label in enumerate(y_pred):
        if label == labels_validation[i]:
            digits_naive_accuracy += 1

digits_naive_accuracy /= len(digits.data)

print(f"Naive base digits validation accuracy: {digits_naive_accuracy*100:.1f}%")

wine_naive_accuracy = 0

for fold in range(5):
    labels_training = np.zeros(len(yWineTrainingSets[fold]))
    
    for i, arr in enumerate(yWineTrainingSets[fold]):
        labels_training[i] = np.argmax(arr)
        
    labels_validation = np.zeros(len(yWineValidationSets[fold]))
    
    for i, arr in enumerate(yWineValidationSets[fold]):
        labels_validation[i] = np.argmax(arr)
    
    gnb = GaussianNB()
    y_pred = gnb.fit(np.asarray(xWineTrainingSets[fold]), labels_training).predict(np.asarray(xWineValidationSets[fold]))

    for i, label in enumerate(y_pred):
        if label == labels_validation[i]:
            wine_naive_accuracy += 1

wine_naive_accuracy /= len(wine.data)

print(f"Naive base wine validation accuracy: {wine_naive_accuracy*100:.1f}%")

Naive base digits validation accuracy: 81.1%
Naive base wine validation accuracy: 93.8%


In [13]:
def HoGFeatures(img, cellSize, blockSize, nbins):
    cell_size = (cellSize, cellSize)
    block_size = (blockSize, blockSize)
    
    hog = cv.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                     img.shape[0] // cell_size[0] * cell_size[0]),
                           _blockSize=(block_size[1] * cell_size[1],
                                       block_size[0] * cell_size[0]),
                           _blockStride=(cell_size[1], cell_size[0]),
                           _cellSize=(cell_size[1], cell_size[0]),
                           _nbins=nbins
    )
    
    return hog

In [14]:
def makeHoGFeatures(imageArray):
    HoG = HoGFeatures(imageArray[0], 2, 2, 2)
    features = []
    
    for i, image in enumerate(imageArray):
        features.append(HoG.compute((image*255).astype(np.uint8)))
        
    features = np.array(np.squeeze(features))
    
    return features

In [15]:
digits_svc_accuracy = 0

for fold in range(5):
    numbers_training = []
    numbers_validation = []
    
    for i, number in enumerate(xDigitsTrainingSets[fold]):
        numbers_training.append(np.asarray(number).reshape(8, 8))
        
    for i, number in enumerate(xDigitsValidationSets[fold]):
        numbers_validation.append(np.asarray(number).reshape(8, 8))  
        
    HoGs_training = makeHoGFeatures(np.asarray(numbers_training))
    HoGs_validation = makeHoGFeatures(np.asarray(numbers_validation))

    clf = svm.SVC(gamma='auto', C=100) 
    
    labels_training = np.zeros(len(yDigitsTrainingSets[fold]))
    
    for i, arr in enumerate(yDigitsTrainingSets[fold]):
        labels_training[i] = np.argmax(arr)
        
    labels_validation = np.zeros(len(yDigitsValidationSets[fold]))
    
    for i, arr in enumerate(yDigitsValidationSets[fold]):
        labels_validation[i] = np.argmax(arr)
    
    clf.fit(HoGs_training, labels_training)

    labels_predicted = clf.predict(HoGs_validation)
    
    for i, label in enumerate(labels_predicted):
        if label == labels_validation[i]:
            digits_svc_accuracy += 1

digits_svc_accuracy /= len(digits.data)

print(f"SVC digits validation accuracy: {digits_svc_accuracy*100:.1f}%")

SVC digits validation accuracy: 89.6%


In [16]:
# Analysis of model hyper-parameters

batch_size = 30
learning_rate = 0.04
momentum = 0.2

In [17]:
# Analysis of changing batch size

batch_size_tests = [30, 25, 20, 15, 10, 5]
batch_size_results = []

for test in batch_size_tests:
    batch_size_results.append(runLogisticRegression(test, learning_rate, momentum))

Model hyper-parameters:
	Mini-batch size: 30
	Learning rate: 0.04
	Momentum: 0.2


Digits gradient descent:
	Cross-validation fold 1
		Step 1: new best accuracy of 0.102
		Step 2: new best accuracy of 0.357
		Step 3
		Step 4
		Step 5: new best accuracy of 0.457
		Step 6
		Step 7
		Step 8
		Step 9: new best accuracy of 0.529
		Step 10: new best accuracy of 0.839
		Step 11
		Step 12
		Step 13: new best accuracy of 0.850
		Step 14
		Step 15
		Step 16: new best accuracy of 0.865
		Step 17: new best accuracy of 0.916
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34: new best accuracy of 0.926
		Step 35
		Step 36: new best accuracy of 0.930
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54
		Step 55
		Step 56
		Step 57
		Step 58
		Step 59
		Step

		Step 18
		Step 19: new best accuracy of 0.905
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26: new best accuracy of 0.914
		Step 27: new best accuracy of 0.921
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33: new best accuracy of 0.926
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45: new best accuracy of 0.927
		Step 46: new best accuracy of 0.928
		Step 47: new best accuracy of 0.931
		Step 48: new best accuracy of 0.933
		Step 49: new best accuracy of 0.935
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54
		Step 55
		Step 56: new best accuracy of 0.942
		Step 57
		Step 58: new best accuracy of 0.942
		Step 59: new best accuracy of 0.945
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68
		Step 69: new best accuracy of 0.946
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74: new best accuracy of 0.948
		Step 75
		Step 76
		Step 77
		Step 7

		Step 33
		Step 34
		Step 35
		Step 36: new best accuracy of 0.923
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46: new best accuracy of 0.924
		Step 47: new best accuracy of 0.927
		Step 48
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54
		Step 55
		Step 56
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65
		Step 66
		Step 67: new best accuracy of 0.937
		Step 68
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78
		Step 79
		Step 80
		Step 81
		Step 82
		Step 83
		Step 84: new best accuracy of 0.939
		Step 85
		Step 86
		Step 87: new best accuracy of 0.940
		Step 88
		Step 89
		Step 90
		Step 91
		Step 92
		Step 93
		Step 94
		Step 95
		Step 96
		Step 97
		Step 98
		Step 99: new best accuracy of 0.944
		Step 100
		Step 101
		Step 102
		Step 103
		Step 104
		Step 105
		Step 106
		Step 107
		Step 108
		Step 109
		Step 110
		Step 111
		

	Cross-validation fold 3
		Step 1: new best accuracy of 0.336
		Step 2: new best accuracy of 0.413
		Step 3
		Step 4: new best accuracy of 0.531
		Step 5
		Step 6
		Step 7
		Step 8: new best accuracy of 0.580
		Step 9: new best accuracy of 0.965
		Step 10
		Step 11
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16
		Step 17
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
	Cross-validation fold 4
		Step 1: new best accuracy of 0.420
		Step 2
		Step 3
		Step 4
		Step 5
		Step 6: new best accuracy of 0.678
		Step 7: new best accuracy of 0.937
		Step 8
		Step 9
		Step 10
		Step 11
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16
		Step 17
		Step 18: new best accuracy of 0.944
		Step 19
		Step 20
		Step 21: new best accuracy of 0.951
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 3

		Step 56
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68
		Step 69
		Step 70
	Cross-validation fold 4
		Step 1: new best accuracy of 0.123
		Step 2: new best accuracy of 0.275
		Step 3
		Step 4: new best accuracy of 0.468
		Step 5: new best accuracy of 0.508
		Step 6
		Step 7
		Step 8
		Step 9
		Step 10: new best accuracy of 0.567
		Step 11: new best accuracy of 0.630
		Step 12: new best accuracy of 0.651
		Step 13: new best accuracy of 0.668
		Step 14: new best accuracy of 0.723
		Step 15
		Step 16: new best accuracy of 0.727
		Step 17: new best accuracy of 0.796
		Step 18: new best accuracy of 0.852
		Step 19
		Step 20
		Step 21
		Step 22: new best accuracy of 0.856
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30: new best accuracy of 0.885
		Step 31
		Step 32: new best accuracy of 0.886
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37: new best accuracy of 0.906
		Step 38
		Step

		Step 87
		Step 88
		Step 89
		Step 90


Digits training accuracy: 93.7%
Digits training cost: 1479.973
Digits validation accuracy: 89.6%
Digits validation cost: 1528.272
Wine training accuracy: 95.1%
Wine training cost: 155.289
Wine validation accuracy: 80.3%
Wine validation cost: 167.310
Model hyper-parameters:
	Mini-batch size: 15
	Learning rate: 0.04
	Momentum: 0.2


Digits gradient descent:
	Cross-validation fold 1
		Step 1: new best accuracy of 0.341
		Step 2
		Step 3
		Step 4
		Step 5: new best accuracy of 0.364
		Step 6: new best accuracy of 0.438
		Step 7: new best accuracy of 0.596
		Step 8
		Step 9
		Step 10
		Step 11: new best accuracy of 0.626
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16: new best accuracy of 0.734
		Step 17
		Step 18
		Step 19: new best accuracy of 0.751
		Step 20
		Step 21
		Step 22
		Step 23: new best accuracy of 0.851
		Step 24: new best accuracy of 0.859
		Step 25: new best accuracy of 0.892
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		St

		Step 31
		Step 32: new best accuracy of 0.905
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44: new best accuracy of 0.911
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54: new best accuracy of 0.918
		Step 55
		Step 56
		Step 57
		Step 58
		Step 59
		Step 60: new best accuracy of 0.922
		Step 61
		Step 62: new best accuracy of 0.929
		Step 63: new best accuracy of 0.932
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78
		Step 79
		Step 80: new best accuracy of 0.933
		Step 81: new best accuracy of 0.934
		Step 82
		Step 83
		Step 84
		Step 85
		Step 86: new best accuracy of 0.939
		Step 87
		Step 88
		Step 89
		Step 90
		Step 91
		Step 92
		Step 93
		Step 94
		Step 95
		Step 96
		Step 97
		Step 98
		Step 99
		Step 100
		Step 101
		Step 102
		Step 103
		Step 104
		S

		Step 66
		Step 67
		Step 68
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78
		Step 79
		Step 80
		Step 81
		Step 82
		Step 83
		Step 84
	Cross-validation fold 3
		Step 1: new best accuracy of 0.102
		Step 2: new best accuracy of 0.147
		Step 3: new best accuracy of 0.366
		Step 4
		Step 5
		Step 6
		Step 7
		Step 8
		Step 9
		Step 10
		Step 11: new best accuracy of 0.517
		Step 12
		Step 13
		Step 14: new best accuracy of 0.528
		Step 15
		Step 16
		Step 17
		Step 18: new best accuracy of 0.729
		Step 19: new best accuracy of 0.805
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36: new best accuracy of 0.833
		Step 37
		Step 38
		Step 39: new best accuracy of 0.836
		Step 40: new best accuracy of 0.862
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49: new best accuracy of 0.

		Step 26
		Step 27
		Step 28: new best accuracy of 0.923
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51: new best accuracy of 0.937
		Step 52
		Step 53
		Step 54
		Step 55
		Step 56
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63: new best accuracy of 0.944
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78
		Step 79
		Step 80
		Step 81
		Step 82: new best accuracy of 0.958
		Step 83
		Step 84
		Step 85
		Step 86
		Step 87
		Step 88
		Step 89
		Step 90
		Step 91
		Step 92
		Step 93
		Step 94
		Step 95
		Step 96
		Step 97
		Step 98
		Step 99
		Step 100
		Step 101
		Step 102
		Step 103
		Step 104
		Step 105
		Step 106
		Step 107
	Cross-validation fold 5
		Step 1: new best accuracy of 0.41

		Step 24
		Step 25
		Step 26: new best accuracy of 0.811
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41: new best accuracy of 0.818
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48: new best accuracy of 0.846
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54
		Step 55
		Step 56
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
	Cross-validation fold 2
		Step 1: new best accuracy of 0.245
		Step 2: new best accuracy of 0.420
		Step 3: new best accuracy of 0.629
		Step 4
		Step 5
		Step 6
		Step 7
		Step 8
		Step 9
		Step 10
		Step 11
		Step 12
		Step 13: new best accuracy of 0.727
		Step 14
		Step 15
		Step 16
		Step 17
		Step 18
		Step 19: new best accuracy of 0.734
		Step 20: new best accuracy of 0.741
		Step 21
		Step 22
		Step 23
		

In [18]:
# Analysis of changing learning rate

learning_rate_tests = [0.02, 0.04, 0.06, 0.08]
learning_rate_results = []

for test in learning_rate_tests:
    learning_rate_results.append(runLogisticRegression(batch_size, test, momentum))

Model hyper-parameters:
	Mini-batch size: 30
	Learning rate: 0.02
	Momentum: 0.2


Digits gradient descent:
	Cross-validation fold 1
		Step 1: new best accuracy of 0.101
		Step 2: new best accuracy of 0.212
		Step 3: new best accuracy of 0.241
		Step 4
		Step 5: new best accuracy of 0.398
		Step 6
		Step 7: new best accuracy of 0.439
		Step 8: new best accuracy of 0.575
		Step 9
		Step 10: new best accuracy of 0.636
		Step 11: new best accuracy of 0.639
		Step 12: new best accuracy of 0.668
		Step 13
		Step 14
		Step 15: new best accuracy of 0.736
		Step 16: new best accuracy of 0.754
		Step 17: new best accuracy of 0.786
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22: new best accuracy of 0.864
		Step 23: new best accuracy of 0.894
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37: new best accuracy of 0.896
		Step 38
		Step 39: new best accuracy of 0.910
		Step 40
		Step 41
		Step 42
		Step 4

		Step 107
		Step 108
		Step 109
		Step 110
		Step 111
		Step 112
		Step 113
		Step 114
		Step 115: new best accuracy of 0.930
		Step 116
		Step 117
		Step 118
		Step 119
		Step 120
		Step 121
		Step 122
		Step 123
		Step 124
		Step 125
		Step 126
		Step 127: new best accuracy of 0.931
		Step 128: new best accuracy of 0.932
		Step 129
		Step 130: new best accuracy of 0.933
		Step 131: new best accuracy of 0.933
		Step 132
		Step 133: new best accuracy of 0.936
		Step 134
		Step 135
		Step 136
		Step 137
		Step 138
		Step 139
		Step 140
		Step 141: new best accuracy of 0.939
		Step 142
		Step 143
		Step 144
		Step 145
		Step 146
		Step 147
		Step 148
		Step 149
		Step 150
		Step 151
		Step 152
		Step 153
		Step 154: new best accuracy of 0.942
		Step 155
		Step 156
		Step 157
		Step 158
		Step 159
		Step 160
		Step 161
		Step 162
		Step 163
		Step 164
		Step 165
		Step 166
		Step 167
		Step 168
		Step 169
		Step 170
		Step 171
		Step 172
		Step 173
		Step 174
		Step 175
		Step 176
		Step

		Step 79
		Step 80
		Step 81
		Step 82
		Step 83
		Step 84
		Step 85
		Step 86
		Step 87
		Step 88
		Step 89: new best accuracy of 0.949
		Step 90
		Step 91
		Step 92
		Step 93
		Step 94: new best accuracy of 0.949
		Step 95
		Step 96
		Step 97
		Step 98
		Step 99
		Step 100: new best accuracy of 0.951
		Step 101
		Step 102
		Step 103
		Step 104
		Step 105
		Step 106: new best accuracy of 0.953
		Step 107
		Step 108
		Step 109
		Step 110
		Step 111
		Step 112
		Step 113
		Step 114
		Step 115
		Step 116
		Step 117
		Step 118
		Step 119
		Step 120
		Step 121
		Step 122
		Step 123
		Step 124
		Step 125
		Step 126
		Step 127
		Step 128
		Step 129
		Step 130
		Step 131
	Cross-validation fold 3
		Step 1: new best accuracy of 0.099
		Step 2: new best accuracy of 0.138
		Step 3: new best accuracy of 0.225
		Step 4: new best accuracy of 0.493
		Step 5: new best accuracy of 0.648
		Step 6
		Step 7
		Step 8: new best accuracy of 0.662
		Step 9
		Step 10: new best accuracy of 0.738
		Step 11
		St

		Step 56
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68: new best accuracy of 0.965
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77: new best accuracy of 0.972
		Step 78
		Step 79
		Step 80
		Step 81
		Step 82
		Step 83
		Step 84
		Step 85
		Step 86
		Step 87
		Step 88
		Step 89
		Step 90
		Step 91
		Step 92
		Step 93
		Step 94
		Step 95: new best accuracy of 0.979
		Step 96
		Step 97
		Step 98
		Step 99
		Step 100
		Step 101
		Step 102
		Step 103
		Step 104
		Step 105
		Step 106
		Step 107
		Step 108
		Step 109
		Step 110
		Step 111
		Step 112
		Step 113
		Step 114
		Step 115
		Step 116
		Step 117
		Step 118
		Step 119
		Step 120
	Cross-validation fold 4
		Step 1: new best accuracy of 0.413
		Step 2
		Step 3
		Step 4: new best accuracy of 0.867
		Step 5
		Step 6
		Step 7
		Step 8
		Step 9
		Step 10
		Step 11
		Step 12
		Step 13: new best accuracy of 0.937
		Step 14
		Ste

		Step 11: new best accuracy of 0.711
		Step 12: new best accuracy of 0.780
		Step 13
		Step 14: new best accuracy of 0.810
		Step 15
		Step 16
		Step 17: new best accuracy of 0.834
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24: new best accuracy of 0.860
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33: new best accuracy of 0.900
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38: new best accuracy of 0.912
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54: new best accuracy of 0.915
		Step 55
		Step 56
		Step 57: new best accuracy of 0.921
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65: new best accuracy of 0.924
		Step 66
		Step 67
		Step 68
		Step 69: new best accuracy of 0.933
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78: new best ac

		Step 17
		Step 18
		Step 19: new best accuracy of 0.846
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25: new best accuracy of 0.928
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
	Cross-validation fold 2
		Step 1: new best accuracy of 0.104
		Step 2: new best accuracy of 0.207
		Step 3: new best accuracy of 0.349
		Step 4: new best accuracy of 0.405
		Step 5
		Step 6
		Step 7: new best accuracy of 0.498
		Step 8
		Step 9
		Step 10
		Step 11
		Step 12: new best accuracy of 0.523
		Step 13: new best accuracy of 0.839
		Step 14
		Step 15
		Step 16
		Step 17
		Step 18
		Step 19: new best accuracy of 0.897
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33: new best accuracy of 0.908
		Step

		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39: new best accuracy of 0.965
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54
		Step 55
		Step 56
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
	Cross-validation fold 4
		Step 1: new best accuracy of 0.413
		Step 2
		Step 3
		Step 4: new best accuracy of 0.573
		Step 5
		Step 6
		Step 7: new best accuracy of 0.664
		Step 8
		Step 9
		Step 10
		Step 11
		Step 12
		Step 13: new best accuracy of 0.678
		Step 14: new best accuracy of 0.706
		Step 15
		Step 16
		Step 17
		Step 18
		Step 19: new best accuracy of 0.944
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30: new best accuracy of 0.958
		Step 31


In [19]:
# Analysis of changing momentum

momentum_tests = [0.2, 0.4, 0.6, 0.8, 0.9]
momentum_results = []

for test in momentum_tests:
    momentum_results.append(runLogisticRegression(batch_size, learning_rate, test))

Model hyper-parameters:
	Mini-batch size: 30
	Learning rate: 0.04
	Momentum: 0.2


Digits gradient descent:
	Cross-validation fold 1
		Step 1: new best accuracy of 0.111
		Step 2
		Step 3: new best accuracy of 0.146
		Step 4: new best accuracy of 0.218
		Step 5: new best accuracy of 0.430
		Step 6: new best accuracy of 0.660
		Step 7
		Step 8: new best accuracy of 0.672
		Step 9
		Step 10: new best accuracy of 0.713
		Step 11: new best accuracy of 0.789
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16
		Step 17: new best accuracy of 0.834
		Step 18
		Step 19: new best accuracy of 0.880
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25: new best accuracy of 0.890
		Step 26
		Step 27
		Step 28: new best accuracy of 0.903
		Step 29
		Step 30
		Step 31: new best accuracy of 0.905
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39: new best accuracy of 0.906
		Step 40
		Step 41
		Step 42: new best accuracy of 0.925
		Step 43
		Step 44
		Step 45
		Step

		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78
		Step 79
		Step 80
		Step 81
		Step 82
		Step 83
		Step 84
		Step 85
		Step 86
		Step 87
		Step 88
		Step 89
		Step 90
Wine gradient descent:
	Cross-validation fold 1
		Step 1: new best accuracy of 0.497
		Step 2
		Step 3: new best accuracy of 0.545
		Step 4
		Step 5: new best accuracy of 0.790
		Step 6
		Step 7
		Step 8: new best accuracy of 0.818
		Step 9
		Step 10: new best accuracy of 0.839
		Step 11
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16
		Step 17
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27: new best accuracy of 0.930
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34: new best accuracy of 0.944
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40: new best accuracy of 0.958
		Step 41: new best accuracy of 0.965
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51

		Step 22
		Step 23
		Step 24
		Step 25
		Step 26: new best accuracy of 0.896
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33: new best accuracy of 0.905
		Step 34: new best accuracy of 0.908
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40: new best accuracy of 0.911
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45: new best accuracy of 0.926
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54: new best accuracy of 0.927
		Step 55
		Step 56
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78
		Step 79
	Cross-validation fold 4
		Step 1: new best accuracy of 0.104
		Step 2: new best accuracy of 0.321
		Step 3: new best accuracy of 0.587
		Step 4
		Step 5
		Step 6
		Step 7: new best accuracy of 0.723
		Step 8
		Step 9
		Step 10
		Step 11
		Step 12: new

		Step 3: new best accuracy of 0.545
		Step 4
		Step 5
		Step 6: new best accuracy of 0.609
		Step 7: new best accuracy of 0.677
		Step 8
		Step 9: new best accuracy of 0.755
		Step 10
		Step 11
		Step 12
		Step 13
		Step 14: new best accuracy of 0.772
		Step 15: new best accuracy of 0.881
		Step 16
		Step 17
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26: new best accuracy of 0.896
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33: new best accuracy of 0.900
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42: new best accuracy of 0.902
		Step 43
		Step 44: new best accuracy of 0.910
		Step 45
		Step 46
		Step 47
		Step 48: new best accuracy of 0.910
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54
		Step 55
		Step 56: new best accuracy of 0.912
		Step 57
		Step 58
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65: new best accuracy of 0.912
		Step 66
		S

		Step 54
		Step 55
		Step 56
		Step 57
		Step 58: new best accuracy of 0.951
		Step 59
		Step 60
		Step 61
		Step 62
		Step 63
		Step 64
		Step 65
		Step 66
		Step 67
		Step 68
		Step 69
		Step 70
		Step 71
		Step 72
		Step 73
		Step 74
		Step 75
		Step 76
		Step 77
		Step 78
		Step 79
		Step 80
		Step 81
		Step 82
		Step 83
	Cross-validation fold 3
		Step 1: new best accuracy of 0.427
		Step 2
		Step 3: new best accuracy of 0.629
		Step 4: new best accuracy of 0.706
		Step 5: new best accuracy of 0.748
		Step 6
		Step 7
		Step 8
		Step 9
		Step 10: new best accuracy of 0.762
		Step 11
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16: new best accuracy of 0.853
		Step 17
		Step 18: new best accuracy of 0.909
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
	Cross-validation fold 4
		Ste

		Step 1: new best accuracy of 0.420
		Step 2
		Step 3
		Step 4
		Step 5: new best accuracy of 0.776
		Step 6
		Step 7
		Step 8
		Step 9
		Step 10
		Step 11
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16
		Step 17
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
	Cross-validation fold 3
		Step 1: new best accuracy of 0.413
		Step 2
		Step 3
		Step 4
		Step 5: new best accuracy of 0.503
		Step 6: new best accuracy of 0.741
		Step 7
		Step 8
		Step 9
		Step 10
		Step 11: new best accuracy of 0.748
		Step 12
		Step 13
		Step 14
		Step 15
		Step 16
		Step 17
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25
		Step 26
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
	Cross-validation fold 4
		Step 1: new best accuracy of 0.413
		Step 2
		Step 3
		Step 4
		Step 5
		Step 6
		Step 7
		Step 8
		Step 9: new best accuracy of 0.629
		S

		Step 17: new best accuracy of 0.685
		Step 18: new best accuracy of 0.743
		Step 19
		Step 20
		Step 21: new best accuracy of 0.782
		Step 22: new best accuracy of 0.856
		Step 23: new best accuracy of 0.859
		Step 24: new best accuracy of 0.897
		Step 25
		Step 26: new best accuracy of 0.900
		Step 27
		Step 28
		Step 29
		Step 30
		Step 31
		Step 32
		Step 33
		Step 34
		Step 35
		Step 36
		Step 37
		Step 38
		Step 39
		Step 40
		Step 41
		Step 42
		Step 43
		Step 44
		Step 45
		Step 46
		Step 47
		Step 48
		Step 49
		Step 50
		Step 51
Wine gradient descent:
	Cross-validation fold 1
		Step 1: new best accuracy of 0.497
		Step 2
		Step 3
		Step 4
		Step 5
		Step 6
		Step 7
		Step 8
		Step 9
		Step 10
		Step 11: new best accuracy of 0.503
		Step 12: new best accuracy of 0.601
		Step 13
		Step 14
		Step 15
		Step 16: new best accuracy of 0.706
		Step 17
		Step 18
		Step 19
		Step 20
		Step 21
		Step 22
		Step 23
		Step 24
		Step 25: new best accuracy of 0.734
		Step 26
		Step 27
		Ste